In [1]:
import numpy as np
rng = np.random.RandomState(42)
x = rng.rand(1000000)
y = rng.rand(1000000)

%timeit x + y

1.34 ms ± 42 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
mask = (x > 0.5) & (y < 0.5)
import numexpr
### Numexpr evaluates the expression in a way that does not use full-sized temporary array
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr) # two arrays are element-wise equal 

True

## `pandas.eval()` for Efficient Operations

In [8]:
import pandas as pd
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

In [9]:
%timeit df1 + df2 + df3 + df4

68.9 ms ± 862 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit pd.eval('df1 + df2 + df3 + df4')

31.3 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))


In [13]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

## including chained expressions

In [14]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

In [15]:
result3 = pd.eval('(df1 < df2) & (df2 <= df3) & (df3 != df4)')

In [16]:
np.allclose(result1, result3)

True

In [17]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

In [20]:
result2.head()

,0,1,2
0,False,True,False
1,False,True,True
2,False,False,True
3,False,True,True
4,False,True,True


In [23]:
df2.T[0]

0     75
1     15
2    719
Name: 0, dtype: int64

In [24]:
df3.iloc[1]

0    766
1    714
2    218
Name: 1, dtype: int64

In [25]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

In [26]:
df2.shape

(100, 3)

## `DataFrame.eval()` for Column-Wise Operations

In [27]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [28]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

In [29]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [38]:
column_mean = df.mean(1) # row means
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)


True

In [37]:
df.mean(0)

A    0.503264
B    0.520534
C    0.502907
D    9.947894
dtype: float64

## `DataFrame.query()` Method -- filtering operation

In [39]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)


True

In [40]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)


True

## accepts the `@` flag to mark local variables

In [42]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean & B < @Cmean')
np.allclose(result1, result2)

True

## check size

In [43]:
df.values.nbytes

32000